In [1]:
import numpy as np
import matplotlib as plt
%matplotlib inline
import copy

In [3]:
board = np.zeros(9)
print(board)

[0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [4]:
def random_set(board,type = 1):
    self_board = board
    while(1):
        index = np.random.randint(0,9)
        if self_board[index] == 0:
            self_board[index] = type
            break
    return self_board

In [5]:
for i in range(9):
    if i % 2 == 0:
        board = random_set(board,2)
    else:
        board = random_set(board,1)

print(board)

[2. 2. 2. 1. 1. 2. 1. 2. 1.]


In [6]:
'''board = np.zeros(9)
bit_ary = [1,2,4,8,16,32,64,128,256]
check_ary = [7,56,448,73,146,292,273,84] 
player1_operation = []
player2_operation = []

def checker(operation):
    for check in check_ary:
        if((check & operation) == check):
            return True
    return False

for i in range(9):
    if i % 2 == 0:
        board = random_set(board,2)
    else:
        board = random_set(board,1)
    
    if checker()*/'''

'board = np.zeros(9)\nbit_ary = [1,2,4,8,16,32,64,128,256]\ncheck_ary = [7,56,448,73,146,292,273,84] \nplayer1_operation = []\nplayer2_operation = []\n\ndef checker(operation):\n    for check in check_ary:\n        if((check & operation) == check):\n            return True\n    return False\n\nfor i in range(9):\n    if i % 2 == 0:\n        board = random_set(board,2)\n    else:\n        board = random_set(board,1)\n    \n    if checker()*/'

In [2]:
class sanmoku:
    bit_ary = [1,2,4,8,16,32,64,128,256]
    check_ary = [7,56,448,73,146,292,273,84] 

    def __init__(self):
        self.board = np.zeros(9)
        self.player1_operation = 0
        self.player2_operation = 0

    def checker(self, type = 1):
        if type == 1:
            operation = self.player1_operation
        else:
            operation = self.player2_operation
        for check in self.check_ary:
            if((check & operation) == check):
                return True
        return False

    def random_set(self, type = 1):
        if np.any(self.board == 0):
            while True:
                index = np.random.randint(0,9)
                if self.board[index] == 0:
                    break
            self.board[index] = type
            if type == 1:
                self.player1_operation += self.bit_ary[index]
            else:
                self.player2_operation += self.bit_ary[index]

    def set(self, index, type = 1):
        self.board[index] = type
        if type == 1:
            self.player1_operation += self.bit_ary[index]
        else:
            self.player2_operation += self.bit_ary[index]

    def step(self, action, type = 1):
        self.set(action, type)
        if type == 1:
            un_type = 2
        else:
            un_type = 1
        if self.checker(type) == False:
            self.random_set(un_type)
        return self.board, self.checker(type), self.checker(un_type)

    def reset(self):
        self.board = np.zeros(9)
        self.player1_operation = 0
        self.player2_operation = 0
        return self.board
    

In [8]:
env = sanmoku()
user = 1
for i in range(9):
    env.random_set(user)
    if env.checker(user):
        break
    if user == 1:
        user = 2
    else:
        user = 1
if i == 8:
    print("draw")
else:
    print("winner:" + str(user))
print(env.board.reshape(3,3))


draw
[[2. 1. 1.]
 [2. 1. 2.]
 [1. 2. 1.]]


In [3]:
GAMMA = 0.99
ETA = 0.5
MAX_STEPS = 9
NUM_EPISODES = 200
NUM_REPETITION = 1000

In [4]:
class Agent:
    def __init__(self, num_states, num_actions):
        self.brain = Brain(num_states, num_actions)

    def update_Q_function(self, observation, action, reward, observation_next):
        self.brain.update_Q_table(
            observation, action, reward, observation_next)

    def get_action(self, observation, step):
        action = self.brain.decide_action(observation, step)
        return action

In [19]:
class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions
        self.q_table = np.random.uniform(low=0, high=1, size=(num_states, num_actions))

    def observation_to_state(self, observation):
        return int(sum([x * (3**i) for i, x in enumerate(observation)]))

    def update_Q_table(self, observation, action, reward, observation_next):
        state = self.observation_to_state(observation)
        state_next = self.observation_to_state(observation_next)
        Max_Q_next = max(self.q_table[state_next][:])
        self.q_table[state, action] = self.q_table[state, action] + \
            ETA * (reward + GAMMA * Max_Q_next - self.q_table[state, action])

    def decide_action(self, observation, episode):
        state = self.observation_to_state(observation)
        epsilon = 0.5 * (1 / (episode + 1))

        if epsilon <= np.random.uniform(0, 1):
            action = np.argmax(self.deduplication(observation, self.q_table[state][:]))
        else:
            action = np.random.choice(self.num_actions)
        return action

    def deduplication(self, observation, q_table):
        bool_not_observation = np.logical_not(observation > 0)
        mask = bool_not_observation.astype(int)
        return q_table * mask

In [21]:
class Environment:
    def __init__(self):
        self.env = sanmoku()
        num_states = 3**9
        num_actions = 9

        self.num_win = 0
        self.num_lose = 0
        
        self.agent = Agent(num_states, num_actions)

    def run(self):
        complete_episodes = 0
        is_episode_final = False

        for episode in range(NUM_EPISODES):
            observation = self.env.reset()

            for step in range(MAX_STEPS):
                action = self.agent.get_action(observation, episode)
                observation_next, done, un_done = self.env.step(action)

                if done:
                    reward = 1
                    complete_episodes += 1
                    self.num_win += 1
                elif un_done:
                    reward = -1
                    complete_episodes = 0
                    self.num_lose += 1
                else:
                    reward = 0

                self.agent.update_Q_function(observation, action, reward, observation_next)

                observation = observation_next

                if done:
                    print('{0} Episode : AI won and Finished after {1} time steps'.format(episode, step + 1))
                    break
                if un_done:
                    print('{0} Episode : AI lost and Finished after {1} time steps'.format(episode, step + 1))
                    break

            if not done or un_done:
                print('{0} Episode : Draw'.format(episode))
            if complete_episodes >= 10:
                print("10連続勝利")
                is_episode_final = True

            if is_episode_final is True:
                break
        
        print("Win rate:{0}".format(self.num_win / (self.num_win + self.num_lose)))

In [25]:
sanmoku_env = Environment()
sanmoku_env.run()

0 Episode : AI lost and Finished after 5 time steps
0 Episode : Draw
1 Episode : AI lost and Finished after 4 time steps
1 Episode : Draw
2 Episode : AI won and Finished after 4 time steps
3 Episode : AI won and Finished after 6 time steps
4 Episode : AI won and Finished after 5 time steps
5 Episode : AI lost and Finished after 4 time steps
5 Episode : Draw
6 Episode : AI won and Finished after 5 time steps
7 Episode : AI won and Finished after 4 time steps
8 Episode : AI won and Finished after 4 time steps
9 Episode : AI won and Finished after 6 time steps
10 Episode : AI won and Finished after 4 time steps
11 Episode : AI won and Finished after 4 time steps
12 Episode : AI won and Finished after 5 time steps
13 Episode : AI lost and Finished after 3 time steps
13 Episode : Draw
14 Episode : AI won and Finished after 4 time steps
15 Episode : AI lost and Finished after 4 time steps
15 Episode : Draw
16 Episode : AI lost and Finished after 3 time steps
16 Episode : Draw
17 Episode : AI

In [27]:
class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions
        self.q_table = np.random.uniform(low=0, high=1, size=(num_states, num_actions))

    def observation_to_state(self, observation):
        return int(sum([x * (3**i) for i, x in enumerate(observation)]))

    def update_Q_table(self, observation, action, reward, observation_next):
        state = self.observation_to_state(observation)
        state_next = self.observation_to_state(observation_next)
        Max_Q_next = max(self.deduplication(observation, self.q_table[state_next][:]))
        self.q_table[state, action] = self.q_table[state, action] + \
            ETA * (reward + GAMMA * Max_Q_next - self.q_table[state, action])

    def decide_action(self, observation, episode):
        state = self.observation_to_state(observation)
        epsilon = 0.5 * (1 / (episode + 1))

        if epsilon <= np.random.uniform(0, 1):
            action = np.argmax(self.deduplication(observation, self.q_table[state][:]))
        else:
            action = np.random.choice(self.num_actions)
        return action

    def deduplication(self, observation, q_table):
        bool_not_observation = np.logical_not(observation > 0)
        mask = bool_not_observation.astype(int)
        return q_table * mask

In [15]:
sanmoku_env2 = Environment()
sanmoku_env2.run()

0 Episode : AI lost and Finished after 4 time steps
0 Episode : Draw
1 Episode : AI won and Finished after 3 time steps
2 Episode : AI won and Finished after 6 time steps
3 Episode : AI won and Finished after 5 time steps
4 Episode : AI lost and Finished after 3 time steps
4 Episode : Draw
5 Episode : AI won and Finished after 4 time steps
6 Episode : AI won and Finished after 5 time steps
7 Episode : AI won and Finished after 3 time steps
8 Episode : AI lost and Finished after 3 time steps
8 Episode : Draw
9 Episode : AI lost and Finished after 3 time steps
9 Episode : Draw
10 Episode : AI won and Finished after 6 time steps
11 Episode : AI won and Finished after 5 time steps
12 Episode : AI lost and Finished after 3 time steps
12 Episode : Draw
13 Episode : AI lost and Finished after 3 time steps
13 Episode : Draw
14 Episode : AI won and Finished after 5 time steps
15 Episode : AI lost and Finished after 3 time steps
15 Episode : Draw
16 Episode : AI lost and Finished after 3 time st

In [46]:
class Environment:
    def __init__(self):
        self.env = sanmoku()
        num_states = 3**9
        num_actions = 9

        self.num_win = 0
        self.num_lose = 0
        
        self.agent = Agent(num_states, num_actions)

    def reset(self):
        self.env = sanmoku()
        num_states = 3**9
        num_actions = 9
        self.agent = Agent(num_states, num_actions)

    def run(self):
        complete_episodes = 0
        is_episode_final = False

        for episode in range(NUM_EPISODES):
            observation = self.env.reset()

            for step in range(MAX_STEPS):
                action = self.agent.get_action(observation, episode)
                observation_next, done, un_done = self.env.step(action)

                if done:
                    reward = 1
                    complete_episodes += 1
                    self.num_win += 1
                elif un_done:
                    reward = -1
                    complete_episodes = 0
                    self.num_lose += 1
                else:
                    reward = 0

                self.agent.update_Q_function(observation, action, reward, observation_next)

                observation = observation_next

                if done or un_done:
                    break

            if complete_episodes >= 10:
                is_episode_final = True

            if is_episode_final is True:
                break
        
        win_rate = self.num_win / (self.num_win + self.num_lose)
        return win_rate, episode

class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions
        self.q_table = np.random.uniform(low=0, high=1, size=(num_states, num_actions))

    def observation_to_state(self, observation):
        return int(sum([x * (3**i) for i, x in enumerate(observation)]))

    def update_Q_table(self, observation, action, reward, observation_next):
        state = self.observation_to_state(observation)
        state_next = self.observation_to_state(observation_next)
        Max_Q_next = max(self.q_table[state_next][:])
        self.q_table[state, action] = self.q_table[state, action] + \
            ETA * (reward + GAMMA * Max_Q_next - self.q_table[state, action])

    def decide_action(self, observation, episode):
        state = self.observation_to_state(observation)
        epsilon = 0.5 * (1 / (episode + 1))

        if epsilon <= np.random.uniform(0, 1):
            action = np.argmax(self.deduplication(observation, self.q_table[state][:]))
        else:
            action = np.random.choice(self.num_actions)
        return action

    def deduplication(self, observation, q_table):
        bool_not_observation = np.logical_not(observation > 0)
        mask = bool_not_observation.astype(int)
        return q_table * mask

sanmoku_env = Environment()
sanmoku_env_win_rate = np.array([])
sanmoku_env_episode = np.array([])
for i in range(NUM_REPETITION):
    sanmoku_env.reset()
    win_rate, episode = sanmoku_env.run()
    sanmoku_env_win_rate = np.append(sanmoku_env_win_rate, win_rate)
    sanmoku_env_episode = np.append(sanmoku_env_episode, episode)
    
class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions
        self.q_table = np.random.uniform(low=0, high=1, size=(num_states, num_actions))

    def observation_to_state(self, observation):
        return int(sum([x * (3**i) for i, x in enumerate(observation)]))

    def update_Q_table(self, observation, action, reward, observation_next):
        state = self.observation_to_state(observation)
        state_next = self.observation_to_state(observation_next)
        Max_Q_next = max(self.deduplication(observation, self.q_table[state_next][:]))
        self.q_table[state, action] = self.q_table[state, action] + \
            ETA * (reward + GAMMA * Max_Q_next - self.q_table[state, action])

    def decide_action(self, observation, episode):
        state = self.observation_to_state(observation)
        epsilon = 0.5 * (1 / (episode + 1))

        if epsilon <= np.random.uniform(0, 1):
            action = np.argmax(self.deduplication(observation, self.q_table[state][:]))
        else:
            action = np.random.choice(self.num_actions)
        return action

    def deduplication(self, observation, q_table):
        bool_not_observation = np.logical_not(observation > 0)
        mask = bool_not_observation.astype(int)
        return q_table * mask

sanmoku_env2 = Environment()
sanmoku_env2_win_rate = np.array([])
sanmoku_env2_episode = np.array([])
for i in range(NUM_REPETITION):
    sanmoku_env2.reset()
    win_rate, episode = sanmoku_env2.run()
    sanmoku_env2_win_rate = np.append(sanmoku_env2_win_rate, win_rate)
    sanmoku_env2_episode = np.append(sanmoku_env2_episode, episode)

class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions
        self.q_table = np.random.uniform(low=0, high=1, size=(num_states, num_actions))

    def observation_to_state(self, observation):
        return int(sum([x * (3**i) for i, x in enumerate(observation)]))

    def update_Q_table(self, observation, action, reward, observation_next):
        state = self.observation_to_state(observation)
        state_next = self.observation_to_state(observation_next)
        Max_Q_next = max(self.deduplication(observation, self.q_table[state_next][:]))
        if reward == 0:
            self.q_table[state, action] = self.q_table[state, action] + \
                ETA * (reward + GAMMA * Max_Q_next - self.q_table[state, action])
        else:
            self.q_table[state, action] = self.q_table[state, action] + ETA * (reward - self.q_table[state, action])

    def decide_action(self, observation, episode):
        state = self.observation_to_state(observation)
        epsilon = 0.5 * (1 / (episode + 1))

        if epsilon <= np.random.uniform(0, 1):
            action = np.argmax(self.deduplication(observation, self.q_table[state][:]))
        else:
            action = np.random.choice(self.num_actions)
        return action

    def deduplication(self, observation, q_table):
        bool_not_observation = np.logical_not(observation > 0)
        mask = bool_not_observation.astype(int)
        return q_table * mask

sanmoku_env3 = Environment()
sanmoku_env3_win_rate = np.array([])
sanmoku_env3_episode = np.array([])
for i in range(NUM_REPETITION):
    sanmoku_env3.reset()
    win_rate, episode = sanmoku_env3.run()
    sanmoku_env3_win_rate = np.append(sanmoku_env3_win_rate, win_rate)
    sanmoku_env3_episode = np.append(sanmoku_env3_episode, episode)

sanmoku_env_win_rate_mean = np.mean(sanmoku_env_win_rate)
sanmoku_env_win_rate_med = np.median(sanmoku_env_win_rate)
sanmoku_env_win_rate_std = np.std(sanmoku_env_win_rate)
sanmoku_env_episode_mean = np.mean(sanmoku_env_episode)
sanmoku_env_episode_med = np.median(sanmoku_env_episode)
sanmoku_env_episode_std = np.std(sanmoku_env_episode)
sanmoku_env2_win_rate_mean = np.mean(sanmoku_env2_win_rate)
sanmoku_env2_win_rate_med = np.median(sanmoku_env2_win_rate)
sanmoku_env2_win_rate_std = np.std(sanmoku_env2_win_rate)
sanmoku_env2_episode_mean = np.mean(sanmoku_env2_episode)
sanmoku_env2_episode_med = np.median(sanmoku_env2_episode)
sanmoku_env2_episode_std = np.std(sanmoku_env2_episode)
sanmoku_env3_win_rate_mean = np.mean(sanmoku_env3_win_rate)
sanmoku_env3_win_rate_med = np.median(sanmoku_env3_win_rate)
sanmoku_env3_win_rate_std = np.std(sanmoku_env3_win_rate)
sanmoku_env3_episode_mean = np.mean(sanmoku_env3_episode)
sanmoku_env3_episode_med = np.median(sanmoku_env3_episode)
sanmoku_env3_episode_std = np.std(sanmoku_env3_episode)

print("sanmoku勝率")
print("平均:{0:.05f},中央値:{1:.05f},標準偏差:{2:.05f},最大:{3:.05f},最小:{4:.05f}".format(sanmoku_env_win_rate_mean,sanmoku_env_win_rate_med,sanmoku_env_win_rate_std,max(sanmoku_env_win_rate),min(sanmoku_env_win_rate)))
print("sanmoku2勝率")
print("平均:{0:.05f},中央値:{1:.05f},標準偏差:{2:.05f},最大:{3:.05f},最小:{4:.05f}".format(sanmoku_env2_win_rate_mean,sanmoku_env2_win_rate_med,sanmoku_env2_win_rate_std,max(sanmoku_env2_win_rate),min(sanmoku_env2_win_rate)))
print("sanmoku3勝率")
print("平均:{0:.05f},中央値:{1:.05f},標準偏差:{2:.05f},最大:{3:.05f},最小:{4:.05f}".format(sanmoku_env3_win_rate_mean,sanmoku_env3_win_rate_med,sanmoku_env3_win_rate_std,max(sanmoku_env3_win_rate),min(sanmoku_env3_win_rate)))
print("sanmokuエピソード数")
print("平均:{0:.05f},中央値:{1:.05f},標準偏差:{2:.05f},最大:{3:.05f},最小:{4:.05f}".format(sanmoku_env_episode_mean,sanmoku_env_episode_med,sanmoku_env_episode_std,max(sanmoku_env_episode),min(sanmoku_env_episode)))
print("sanmoku2エピソード数")
print("平均:{0:.05f},中央値:{1:.05f},標準偏差:{2:.05f},最大:{3:.05f},最小:{4:.05f}".format(sanmoku_env2_episode_mean,sanmoku_env2_episode_med,sanmoku_env2_episode_std,max(sanmoku_env2_episode),min(sanmoku_env2_episode)))
print("sanmoku3エピソード数")
print("平均:{0:.05f},中央値:{1:.05f},標準偏差:{2:.05f},最大:{3:.05f},最小:{4:.05f}".format(sanmoku_env3_episode_mean,sanmoku_env3_episode_med,sanmoku_env3_episode_std,max(sanmoku_env3_episode),min(sanmoku_env3_episode)))


sanmoku勝率
平均:0.67792,中央値:0.67799,標準偏差:0.00440,最大:0.71371,最小:0.66802
sanmoku2勝率
平均:0.68571,中央値:0.68489,標準偏差:0.00498,最大:0.75159,最小:0.67458
sanmoku3勝率
平均:0.68231,中央値:0.68241,標準偏差:0.00650,最大:0.69059,最小:0.59873
sanmokuエピソード数
平均:98.14200,中央値:80.50000,標準偏差:68.26769,最大:199.00000,最小:9.00000
sanmoku2エピソード数
平均:96.43600,中央値:78.00000,標準偏差:67.89677,最大:199.00000,最小:9.00000
sanmoku3エピソード数
平均:92.71500,中央値:73.50000,標準偏差:67.61171,最大:199.00000,最小:9.00000


In [15]:
import random
NUM_REPETITION = 20000

class Environment:
    def __init__(self):
        self.env = sanmoku()
        num_states = 3**9
        num_actions = 9

        self.num_win = 0
        self.num_lose = 0
        
        self.agent = Agent(num_states, num_actions)

    def reset(self):
        self.env = sanmoku()
        num_states = 3**9
        num_actions = 9
        self.agent = Agent(num_states, num_actions)

    def run(self):
        complete_episodes = 0
        is_episode_final = False

        for episode in range(NUM_EPISODES):
            observation = copy.deepcopy(self.env.reset())
            type = random.randint(1,2)

            for step in range(MAX_STEPS):
                action = self.agent.get_action(observation, episode)
                observation_next, done, un_done = self.env.step(action,type)

                if done:
                    reward = 1
                    complete_episodes += 1
                    self.num_win += 1
                elif un_done:
                    reward = -1
                    complete_episodes = 0
                    self.num_lose += 1
                else:
                    reward = 0

                self.agent.update_Q_function(observation, action, reward, observation_next)

                observation = copy.deepcopy(observation_next)

                if done or un_done:
                    break

            if complete_episodes >= 10:
                is_episode_final = True

            if is_episode_final is True:
                break
        
        win_rate = self.num_win / (self.num_win + self.num_lose)
        return win_rate, episode

class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions
        self.q_table = np.random.uniform(low=0, high=1, size=(num_states, num_actions))

    def observation_to_state(self, observation):
        return int(sum([x * (3**i) for i, x in enumerate(observation)]))

    def update_Q_table(self, observation, action, reward, observation_next):
        state = self.observation_to_state(observation)
        state_next = self.observation_to_state(observation_next)
        Max_Q_next = max(self.deduplication(observation, self.q_table[state_next][:]))
        self.q_table[state, action] = self.q_table[state, action] + \
            ETA * (reward + GAMMA * Max_Q_next - self.q_table[state, action])

    def decide_action(self, observation, episode):
        state = self.observation_to_state(observation)
        epsilon = 0.5 * (1 / (episode + 1))

        if epsilon <= np.random.uniform(0, 1):
            action = np.argmax(self.deduplication(observation, self.q_table[state][:]))
        else:
            action = np.random.choice(self.num_actions)
        return action

    def deduplication(self, observation, q_table):
        bool_not_observation = np.logical_not(observation > 0)
        mask = bool_not_observation.astype(int)
        return q_table * mask

sanmoku_env2 = Environment()
sanmoku_env2_win_rate = np.array([])
sanmoku_env2_episode = np.array([])
for i in range(NUM_REPETITION):
    win_rate, episode = sanmoku_env2.run()
    sanmoku_env2_win_rate = np.append(sanmoku_env2_win_rate, win_rate)
    sanmoku_env2_episode = np.append(sanmoku_env2_episode, episode)

sanmoku_env2_win_rate_mean = np.mean(sanmoku_env2_win_rate)
sanmoku_env2_win_rate_med = np.median(sanmoku_env2_win_rate)
sanmoku_env2_win_rate_std = np.std(sanmoku_env2_win_rate)
sanmoku_env2_episode_mean = np.mean(sanmoku_env2_episode)
sanmoku_env2_episode_med = np.median(sanmoku_env2_episode)
sanmoku_env2_episode_std = np.std(sanmoku_env2_episode)

print("sanmoku2勝率")
print("平均:{0:.05f},中央値:{1:.05f},標準偏差:{2:.05f},最大:{3:.05f},最小:{4:.05f}".format(sanmoku_env2_win_rate_mean,sanmoku_env2_win_rate_med,sanmoku_env2_win_rate_std,max(sanmoku_env2_win_rate),min(sanmoku_env2_win_rate)))
print("sanmoku2エピソード数")
print("平均:{0:.05f},中央値:{1:.05f},標準偏差:{2:.05f},最大:{3:.05f},最小:{4:.05f}".format(sanmoku_env2_episode_mean,sanmoku_env2_episode_med,sanmoku_env2_episode_std,max(sanmoku_env2_episode),min(sanmoku_env2_episode)))


sanmoku2勝率
平均:0.91803,中央値:0.92389,標準偏差:0.01842,最大:0.93152,最小:0.71147
sanmoku2エピソード数
平均:11.79140,中央値:10.00000,標準偏差:4.71893,最大:176.00000,最小:9.00000


In [17]:


class VS_Q_Learning:
    
    def __init__(self):
        num_states = 9
        num_actions = 9
        self.game_board = np.zeros(num_states)
        self.agent = VS_Agent(num_states, num_actions)

    def play(self):
        
        pre = int(input("先攻→0, 後攻→1を入力してください"))
        print("ゲームスタート!")
        self.show()
        #observation = copy.deepcopy(self.game_board)
        winner = 0
            
        for step in range(9):
            
            if pre == 0:
                winner = self.your_turn()
                if (winner == 0 and step < 4):
                    winner = self.q_learning_turn()
            else:
                winner = self.q_learning_turn()
                if (winner == 0 and step < 4):
                    winner = self.your_turn()

            if winner == 1:
                print("AIの勝ち")
                break
            elif winner == 2:
                print("あなたの勝ち")
                break

            if step == 4:
                print("引き分け")
                complete_episodes = 0
                break

    def your_turn(self):
        print("あなたのターン")
        pos = int(input("手を選んでください:"))
        self.game_board[pos] = 2
        self.show()
        winner = self.winner_func(2)
        return winner
        
    def q_learning_turn(self):
        print("AIのターン")
        action = self.agent.get_action(self.game_board)
        self.game_board[action] = 1
        self.show()
        winner = self.winner_func(1)
        return winner

    def winner_func(self, player):
        # 勝ち手を列挙
        lines = [
        [0, 1, 2],
        [3, 4, 5],
        [6, 7, 8],
        [0, 3, 6],
        [1, 4, 7],
        [2, 5, 8],
        [0, 4, 8],
        [2, 4, 6],
        ]
        for i in range(0, len(lines)):
            [a, b, c] = lines[i]
            
            if self.game_board[a] and self.game_board[a] == self.game_board[b] and self.game_board[a] == self.game_board[c]: 
                return player
        return 0

    def show(self):
        show_board = np.array(["","","","","","","","",""])
        for i in range(9):
            x = self.game_board[i]
            if (x==0):
                show_board[i] = str(i)
            elif (x==1):
                show_board[i] = "×"
            else:
                show_board[i] = "︎O"
        board = np.reshape(show_board, [3,3])
        print(board)

class VS_Agent:
    '''三目並べのエージェントクラス'''
    
    def __init__(self, num_states, num_actions):
        self.brain = VS_Brain(num_states, num_actions)
        # エージェントが行動を決定するための頭脳を生成
            
    def get_action(self, observation):
        '''行動の決定'''
        action = self.brain.decide_action(observation)
        return action

class VS_Brain:
    '''エージェントが持つ脳となるクラスです。Q学習を実行します'''
    
    def __init__(self, num_states, num_actions):
        self.num_states = num_states
        self.num_actions = num_actions
    
    def digitize_state(self, observation):
        return int(sum([x * (3**i) for i, x in enumerate(observation)]))
        
    def decide_action(self, observation):
        choices = []
        for i in range(9):
            if (observation[i] == 0):
                choices.append(i)
        state = self.digitize_state(observation)
        action = np.argmax(q_table[state][:])
        if (action not in choices):
            action = random.choice(choices)
        return action

q_table = sanmoku_env2.agent.brain.q_table
game = VS_Q_Learning()
game.play()


ゲームスタート!
[['0' '1' '2']
 ['3' '4' '5']
 ['6' '7' '8']]
あなたのターン
[['︎' '1' '2']
 ['3' '4' '5']
 ['6' '7' '8']]
AIのターン
[['︎' '1' '2']
 ['3' '×' '5']
 ['6' '7' '8']]
あなたのターン
[['︎' '1' '2']
 ['3' '×' '5']
 ['6' '7' '︎']]
AIのターン
[['︎' '×' '2']
 ['3' '×' '5']
 ['6' '7' '︎']]
あなたのターン
[['︎' '×' '2']
 ['3' '×' '︎']
 ['6' '7' '︎']]
AIのターン
[['︎' '×' '2']
 ['3' '×' '︎']
 ['6' '×' '︎']]
AIの勝ち
